In [1]:
# import packages 
import gffutils
from Bio import SeqIO 
import numpy as np
from Bio.Seq import Seq
import pandas as pd

In [2]:
# create database using gtf file(should be the same one as used for rMATS analysis)
fn1='Homo_sapiens.GRCh38.90.gtf.gz'
db = gffutils.create_db(fn1,":memory:",keep_order=True,disable_infer_genes=True, disable_infer_transcripts=True)

In [3]:
# parse ref fasta seq
file_path = "Homo_sapiens.GRCh38.dna.primary_assembly.fa"
hg38_sequences = list(SeqIO.parse(file_path, "fasta"))

In [4]:
# covert csv to txt file, then load txt  
def covReadCSV(csv_path,txt_path):
    data = pd.read_csv(csv_path)
    data.to_csv(txt_path, sep='\t', index=False)
    columns_to_read = list(range(2, 12)) + list(range(18, 21))
    AS = np.loadtxt(txt_path, dtype=str, delimiter='\t', skiprows=1, usecols=columns_to_read)
    return AS

In [20]:
# Function: getA5Frame
# Inputs: rMATs file after processing(DSSEs) and gff database(!Note the release version of gff file, fasta file, and rMATs file should be the same!)  
# Returns: array file containing info of the upstream exon start/end position, frame, target exon start/end position, downstream exon start/end position 
# Summary: retrieve the frame information from gff file for translation 
def getA5Frame(se, db):
    dtype = [('Gene_ID', 'U20'), ('strand', 'U10'), ('CDS_ID', 'U20'), ('CDS_Start', int), ('CDS_Stop', int), ('CDS_Frame', int),('S_Start', int), ('S_Stop', int),('L_Start', int), ('L_Stop', int),('Down_Start', int), ('Down_Stop', int), ('Chrom', 'U20'), ('Gene_name', 'U20'),
            ('subtype', 'U20'), ('type', 'U10'), ('comp', 'U20')]
    gene_frame_array = np.array([], dtype=dtype)
    for l in se:
        gene_id = l[0]
        gene_name = l[1]
        chrom = l[2]
        strand = l[3]
        DownES = int(l[8])
        DownEE = int(l[9])
        LES = int(l[4])
        LEE = int(l[5])
        SES = int(l[6])
        SEE = int(l[7])
        subtype = l[10]
        type = l[11]
        comp = l[12]
        cds_features = db.children(gene_id, featuretype='CDS')
        if strand == '+':
            for cds in cds_features:
                if cds.start == SES+1 and (cds.stop == SEE or cds.stop == LEE):
                    # print(gene_id,cds.id,cds.start,cds.stop,cds.frame)
                    gene_frame_array = np.append(gene_frame_array, np.array([(gene_id, strand, cds.id, cds.start, cds.stop, cds.frame, SES, SEE, LES, LEE, DownES, DownEE, chrom, gene_name, subtype, type, comp)], dtype=dtype))
                    break
                elif (cds.stop == SEE or cds.stop == LEE) and cds.start > SES+1:
                    gene_frame_array = np.append(gene_frame_array, np.array([(gene_id, strand, cds.id, cds.start, cds.stop, cds.frame, SES, SEE, LES, LEE, DownES, DownEE, chrom, gene_name, subtype, type, comp)], dtype=dtype))
                    break 
        else:
            for cds in cds_features:
                if cds.start == SES+1  and (cds.stop == SEE or cds.stop == LEE):
                    # print(gene_id,cds.id,cds.start,cds.stop,cds.frame)
                    gene_frame_array = np.append(gene_frame_array, np.array([(gene_id, strand, cds.id, cds.start, cds.stop, cds.frame, SES, SEE, LES, LEE, DownES, DownEE,chrom, gene_name, subtype, type, comp)], dtype=dtype))
                    break
                elif (cds.stop < SEE or cds.stop < LEE) and cds.start == SES+1:
                    gene_frame_array = np.append(gene_frame_array, np.array([(gene_id, strand, cds.id, cds.start, cds.stop, cds.frame, SES, SEE, LES, LEE, DownES, DownEE,chrom, gene_name, subtype, type, comp)], dtype=dtype))
                    break 
    return gene_frame_array

In [10]:
# Function: getA5Seq
# Inputs: rMATs file after processing(DSSEs) and hg38 reference(!Note the release version of gff file, fasta file, and rMATs file should be the same!)  
# Returns: nucleotide sequence of designated coordinate range
# Summary: retrieve nucleotide sequence based on the coordiate(start and end position of each exon of RI events) 
def getA5Seq(gene_frame_array, hg38_sequences):
    # Define the data type for the structured array
    dtype = [('Gene_ID', 'U20'), ('CDS_Frame', int), ('Chrom', 'U20'), ('Seq', 'U10000'), ('se_Seq', 'U10000'), ('gene_name', 'U20'),('subtype', 'U20'),('type', 'U20'),('comp', 'U20')]  # Adjust the max sequence length as needed
    seq_array = np.array([], dtype=dtype)
    
    # Loop through your data, extract sequences, and add them to the structured array
    for l in gene_frame_array:
        gene = l[0]
        strand = l[1]
        ES = l[3]
        frame = l[5]
        SES = l[6]
        SEE = l[7]
        LES = l[8]
        LEE = l[9]
        DownES = l[10]
        DownEE = l[11]
        chrom = l[12]
        gene_name = l[13]
        subtype = l[14]
        type = l[15]
        comp = l[16]
        
        chrom_id = chrom[3:] # Extract the chromosome ID
    
        # Find the sequence for the specified chromosome
        for record in hg38_sequences:
            if record.id == chrom_id:
                l_seq = record.seq[ES - 1:LEE]
                s_seq = record.seq[ES -1 :SEE]
                down_seq = record.seq[DownES: DownEE]
                seq = str(s_seq + down_seq)  # Convert the sequence to a string
                A5_seq = str(l_seq + down_seq)
               
                if strand == '+':
                    seq_array = np.append(seq_array, np.array([(gene, frame, chrom_id, seq, A5_seq, gene_name,subtype,type, comp)], dtype=dtype))
                    # print(gene, seq)
                    # print(se_seq)
                elif strand == '-':
                    s_seq = record.seq[SES :SEE]
                    l_seq = record.seq[LES :LEE]
                    seq = s_seq[::-1] + down_seq[::-1]
                    seq = str(Seq(seq).complement())
                    A5_seq = l_seq[::-1] + down_seq[::-1]
                    A5_seq = str(Seq(A5_seq).complement())  
                    seq_array = np.append(seq_array, np.array([(gene, frame, chrom_id, seq, A5_seq, gene_name,subtype,type,comp)], dtype=dtype))
    return seq_array


In [11]:
# Function: getAASeq
# Inputs: seq_array 
# Returns: amino acid sequence 
# Summary: translate the nucleotide sequence into amino acid sequence 
def getAASeq(seq_array):
    dtype = [('Gene_ID', 'U20'),('Gene_name', 'U20'), ('aa_seq', 'U10000'), ('se_aa_seq', 'U10000'),('subtype','U20'),('type','U20'),('comp','U20')]  # Adjust the max sequence length as needed
    aa_array = np.array([], dtype=dtype)

    for l in seq_array:
        gene = l[0]
        frame = l[1]
        seq = l[3]
        se_seq = l[4]
        gene_name = l[5]
        subtype = l[6]
        type = l[7]
        comp = l[8]
        aa = Seq(seq[frame:]).translate(to_stop=True)
        se_aa = Seq(se_seq[frame:]).translate(to_stop = True)
        aa_array = np.append(aa_array, np.array([(gene, gene_name, str(aa), str(se_aa),subtype,type,comp)], dtype=dtype))
    return aa_array 

In [18]:
def main():
    a5ss = covReadCSV(a5ss_csv_path,a5ss_txt_path)
    a5_frame = getA5Frame(a5ss, db)
    a5_seq=getA5Seq(a5_frame, hg38_sequences)
    a5_AA_seq = getAASeq(a5_seq)
    file_name = "MPN_neoepitope/A5SS_peptides_MPN.txt"
    fmt = "%s\t%s\t%s\t%s\t%s\t%s\t%s"
    np.savetxt(file_name, a5_AA_seq, fmt=fmt, delimiter='\t')
    print('file saved!')

In [19]:
if __name__ == '__main__':
    a5ss_csv_path = 'MPN_neoepitope/neojunction/A5SS_all_neoj.csv'
    a5ss_txt_path = 'MPN_neoepitope/neojunction/A5SS_all_neoj.txt'
    main()

file saved!


/scratch/prj/kordastilab_neoepitope/gffutil/jvenv/lib/python3.9/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
